In [1]:
!pip install -U llama-index sentence-transformers bitsandbytes llama-index-embeddings-huggingface llama-index-llms-huggingface llama-index

In [2]:
from llama_cloud_services import LlamaExtract
import os
os.environ["LLAMA_CLOUD_API_KEY"] = '<YOUR_API_KEY>'

project_id='<your_project_id>'

def load_extractor(project_id,agent_name):
  llama_extract = LlamaExtract(project_id=project_id)
  agent = llama_extract.get_agent(name="Precise_Resume_extract")
  print(agent.data_schema)
  return agent

extractor = load_extractor(project_id,'Precise_Resume_extract')

{'additionalProperties': False, 'properties': {'basics': {'additionalProperties': False, 'properties': {'name': {'description': 'The full name of the candidate', 'type': 'string'}, 'email': {'description': 'The email address of the candidate', 'type': 'string'}, 'phone': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'The phone number of the candidate in any standard format'}, 'location': {'anyOf': [{'additionalProperties': False, 'description': 'The current location of the candidate', 'properties': {'city': {'description': 'The city where the candidate is located', 'type': 'string'}, 'region': {'description': 'State or province of the candidate', 'type': 'string'}, 'country': {'description': 'Country where the candidate is located', 'type': 'string'}}, 'required': ['city', 'region', 'country'], 'type': 'object'}, {'type': 'null'}]}, 'profiles': {'anyOf': [{'items': {'additionalProperties': False, 'properties': {'network': {'description': 'Name of the social network (

In [3]:
parsed_data = extractor.extract('/content/Backend AI Engineer Resume Template - Claude.pdf')
print(parsed_data.data)

Extracting files: 100%|██████████| 1/1 [00:35<00:00, 35.65s/it]

{'basics': {'name': 'Alexandra Chen', 'email': 'alexandra.chen@email.com', 'phone': '+1 (555) 123-4567', 'location': {'city': 'San Francisco', 'region': 'CA', 'country': 'USA'}, 'profiles': [{'network': 'LinkedIn', 'url': 'https://linkedin.com/in/alexandrachen'}, {'network': 'GitHub', 'url': 'https://github.com/alexchen-dev'}], 'summary': 'Accomplished Senior Backend AI Engineer with 8+ years of experience designing and implementing scalable, high-performance backend systems and AI/ML solutions. Proven track record of leading cross-functional teams to deliver enterprise-grade applications serving millions of users. Expertise in distributed systems, microservices architecture, machine learning pipelines, and cloud-native technologies. Passionate about leveraging cutting-edge AI technologies to solve complex business problems while maintaining system reliability and performance.'}, 'skills': [{'category': 'Programming Languages', 'keywords': ['Python', 'Java', 'Scala', 'Go', 'C++', 'Java

In [4]:
def flatten_json(obj, indent=0):
    lines = []
    spacer = '  ' * indent

    if isinstance(obj, dict):
        for key, value in obj.items():
            key_title = key.replace("_", " ").capitalize()
            if isinstance(value, (dict, list)):
                lines.append(f"{spacer}### {key_title}")
                lines.append("")
                lines.extend(flatten_json(value, indent + 1))
            else:
                lines.append(f"{spacer}- {key_title}: {value}")
    elif isinstance(obj, list):
        for item in obj:
            if isinstance(item, (dict, list)):
                lines.extend(flatten_json(item, indent))  # No "### Item"
                lines.append("")
            else:
                lines.append(f"{spacer}- {item}")
    else:
        lines.append(f"{spacer}- {obj}")
    return lines


def resume_json_to_text(data):
    lines = []
    for section_key, section_value in data.items():
        title = section_key.replace("_", " ").capitalize()
        lines.append(f"## {title}")
        lines.append("")  # blank line after section title
        lines.extend(flatten_json(section_value, indent=1))
        lines.append("")  # blank line between sections
    return "\n".join(lines)


In [5]:
resume_text = resume_json_to_text(parsed_data.data)
print(resume_text)

## Basics

  - Name: Alexandra Chen
  - Email: alexandra.chen@email.com
  - Phone: +1 (555) 123-4567
  ### Location

    - City: San Francisco
    - Region: CA
    - Country: USA
  ### Profiles

    - Network: LinkedIn
    - Url: https://linkedin.com/in/alexandrachen

    - Network: GitHub
    - Url: https://github.com/alexchen-dev

  - Summary: Accomplished Senior Backend AI Engineer with 8+ years of experience designing and implementing scalable, high-performance backend systems and AI/ML solutions. Proven track record of leading cross-functional teams to deliver enterprise-grade applications serving millions of users. Expertise in distributed systems, microservices architecture, machine learning pipelines, and cloud-native technologies. Passionate about leveraging cutting-edge AI technologies to solve complex business problems while maintaining system reliability and performance.

## Skills

  - Category: Programming Languages
  ### Keywords

    - Python
    - Java
    - Scala
    

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
from llama_index.core.schema import Document
from llama_index.core.node_parser import SentenceSplitter,SimpleNodeParser
from llama_index.core import VectorStoreIndex
import re


def extract_sections(text):
    # Find all top-level ## section headers and their positions
    section_pattern = re.compile(r"^(## .+)$", re.MULTILINE)
    matches = list(section_pattern.finditer(text))

    chunks = {}
    for i, match in enumerate(matches):
        section_title = match.group(1).replace('## ', '').strip()
        # Determine section start and end
        section_start = match.end()
        if i + 1 < len(matches):
            section_end = matches[i + 1].start()
        else:
            section_end = len(text)
        section_content = text[section_start:section_end].strip()
        chunks[section_title] = section_content
    return chunks



sections = extract_sections(resume_text)
for title, content in sections.items():
    print(f"\n==== {title} ====\n{content}\n")



==== Basics ====
- Name: Alexandra Chen
  - Email: alexandra.chen@email.com
  - Phone: +1 (555) 123-4567
  ### Location

    - City: San Francisco
    - Region: CA
    - Country: USA
  ### Profiles

    - Network: LinkedIn
    - Url: https://linkedin.com/in/alexandrachen

    - Network: GitHub
    - Url: https://github.com/alexchen-dev

  - Summary: Accomplished Senior Backend AI Engineer with 8+ years of experience designing and implementing scalable, high-performance backend systems and AI/ML solutions. Proven track record of leading cross-functional teams to deliver enterprise-grade applications serving millions of users. Expertise in distributed systems, microservices architecture, machine learning pipelines, and cloud-native technologies. Passionate about leveraging cutting-edge AI technologies to solve complex business problems while maintaining system reliability and performance.


==== Skills ====
- Category: Programming Languages
  ### Keywords

    - Python
    - Java
    - 

In [8]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import Document

def create_nodes_with_custom_chunking(sections_dict):
    all_nodes = []

    for section_name, content in sections_dict.items():
        section_length = len(content)
        print(f"{section_name} section: {section_length} characters")

        # Create section-specific parser
        parser = SentenceSplitter(
            chunk_size=section_length + 200,  # Section size + buffer
            chunk_overlap=0,
            chunking_tokenizer_fn=None
        )

        # Create document for this section
        doc = Document(text=content, metadata={'section': section_name})

        # Parse this section
        section_nodes = parser.get_nodes_from_documents([doc])
        all_nodes.extend(section_nodes)

    return all_nodes

# Use it
nodes = create_nodes_with_custom_chunking(sections)
#Visualize Node Data
for idx,node in enumerate(nodes):
    print(f"\n\n---Node {idx+1}---\n\n")
    print(node.metadata)
    print(node.text)

Basics section: 882 characters
Skills section: 2383 characters
Experience section: 5018 characters
Education section: 326 characters
Certifications section: 519 characters
Publications section: 375 characters


---Node 1---


{'section': 'Basics'}
- Name: Alexandra Chen
  - Email: alexandra.chen@email.com
  - Phone: +1 (555) 123-4567
  ### Location

    - City: San Francisco
    - Region: CA
    - Country: USA
  ### Profiles

    - Network: LinkedIn
    - Url: https://linkedin.com/in/alexandrachen

    - Network: GitHub
    - Url: https://github.com/alexchen-dev

  - Summary: Accomplished Senior Backend AI Engineer with 8+ years of experience designing and implementing scalable, high-performance backend systems and AI/ML solutions. Proven track record of leading cross-functional teams to deliver enterprise-grade applications serving millions of users. Expertise in distributed systems, microservices architecture, machine learning pipelines, and cloud-native technologies. Passionate abou

In [9]:
from llama_index.core import VectorStoreIndex

# Use the nodes from custom parsing
index = VectorStoreIndex(nodes) ##Index from nodes
index.storage_context.persist(persist_dir="resume_index")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
# Basic index information
print(f"Index type: {type(index)}")
print(f"Number of nodes in index: {len(index.docstore.docs)}")
print(f"Index ID: {index.index_id}")


Index type: <class 'llama_index.core.indices.vector_store.base.VectorStoreIndex'>
Number of nodes in index: 6
Index ID: 79b38e08-0167-4e28-b2c5-fc6fd72f2df9


In [11]:
def analyze_index_contents(index):
    """Comprehensive analysis of index contents"""

    print("=== INDEX ANALYSIS ===\n")

    # Basic stats
    total_nodes = len(index.docstore.docs)
    total_chars = sum(len(node.text) for node in index.docstore.docs.values())

    print(f"Total nodes: {total_nodes}")
    print(f"Total characters: {total_chars:,}")
    print(f"Average chars per node: {total_chars/total_nodes:.0f}")

    # Section breakdown
    section_stats = {}
    for node in index.docstore.docs.values():
        section = node.metadata.get('section', 'Unknown')
        if section not in section_stats:
            section_stats[section] = {'count': 0, 'chars': 0}
        section_stats[section]['count'] += 1
        section_stats[section]['chars'] += len(node.text)

    print("\n=== BY SECTION ===")
    for section, stats in section_stats.items():
        print(f"{section}:")
        print(f"  Nodes: {stats['count']}")
        print(f"  Characters: {stats['chars']:,}")
        print(f"  Avg per node: {stats['chars']/stats['count']:.0f}")

    return section_stats

# Run analysis
stats = analyze_index_contents(index)


=== INDEX ANALYSIS ===

Total nodes: 6
Total characters: 9,503
Average chars per node: 1584

=== BY SECTION ===
Basics:
  Nodes: 1
  Characters: 882
  Avg per node: 882
Skills:
  Nodes: 1
  Characters: 2,383
  Avg per node: 2383
Experience:
  Nodes: 1
  Characters: 5,018
  Avg per node: 5018
Education:
  Nodes: 1
  Characters: 326
  Avg per node: 326
Certifications:
  Nodes: 1
  Characters: 519
  Avg per node: 519
Publications:
  Nodes: 1
  Characters: 375
  Avg per node: 375


In [12]:
from llama_index.core import StorageContext, load_index_from_storage

# Load the persisted index
storage_context = StorageContext.from_defaults(persist_dir="resume_index")
loaded_index = load_index_from_storage(storage_context)

# Verify it loaded correctly
print(f"Loaded index has {len(loaded_index.docstore.docs)} nodes")

# Compare with original
print(f"Original had {len(index.docstore.docs)} nodes")
print(f"Match: {len(loaded_index.docstore.docs) == len(index.docstore.docs)}")


Loading llama_index.core.storage.kvstore.simple_kvstore from resume_index/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from resume_index/index_store.json.
Loaded index has 6 nodes
Original had 6 nodes
Match: True


In [13]:
import os

# Check what was persisted
persist_dir = "resume_index"
if os.path.exists(persist_dir):
    files = os.listdir(persist_dir)
    print(f"Files in {persist_dir}:")
    for file in files:
        file_path = os.path.join(persist_dir, file)
        size = os.path.getsize(file_path)
        print(f"  {file} ({size} bytes)")


Files in resume_index:
  graph_store.json (18 bytes)
  default__vector_store.json (52876 bytes)
  image__vector_store.json (72 bytes)
  docstore.json (16187 bytes)
  index_store.json (751 bytes)


In [14]:
# See all node IDs
node_ids = list(index.docstore.docs.keys())
print(f"Node IDs: {node_ids[:5]}...")  # Show first 5

# Examine individual nodes
for i, node_id in enumerate(node_ids):  # Show first 3 nodes
    node = index.docstore.docs[node_id]
    print(f"\n--- Node {i+1} ---")
    print(f"Node ID: {node_id}")
    print(f"Metadata: {node.metadata}")
    print(f"Text preview: {node.text[:200]}...")  # First 200 chars
    print(f"Text length: {len(node.text)} characters")


Node IDs: ['b49e6320-85fd-45a9-80e5-9017d9fda27d', 'd9ab67a8-d6cc-4d36-84fc-530ad80e8aa5', '9eb674be-6c9e-43fe-b462-eb9626b27af0', 'a6dce4d3-159e-4f55-8a9c-b6f5dbe0a88f', 'f2a4e0c6-997d-434c-89aa-f6b407d1e4d8']...

--- Node 1 ---
Node ID: b49e6320-85fd-45a9-80e5-9017d9fda27d
Metadata: {'section': 'Basics'}
Text preview: - Name: Alexandra Chen
  - Email: alexandra.chen@email.com
  - Phone: +1 (555) 123-4567
  ### Location

    - City: San Francisco
    - Region: CA
    - Country: USA
  ### Profiles

    - Network: Lin...
Text length: 882 characters

--- Node 2 ---
Node ID: d9ab67a8-d6cc-4d36-84fc-530ad80e8aa5
Metadata: {'section': 'Skills'}
Text preview: - Category: Programming Languages
  ### Keywords

    - Python
    - Java
    - Scala
    - Go
    - C++
    - JavaScript/TypeScript
    - SQL
    - R
  - Level: expert

  - Category: AI/ML Technologi...
Text length: 2383 characters

--- Node 3 ---
Node ID: 9eb674be-6c9e-43fe-b462-eb9626b27af0
Metadata: {'section': 'Experience'}
Text pr

In [15]:
# !huggingface-cli login

In [16]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core import get_response_synthesizer, PromptTemplate
from transformers import BitsAndBytesConfig
import torch

model_name = "google/gemma-7b-it"
memory = ChatMemoryBuffer.from_defaults(token_limit=2048)

def load_model(model_name):
    """Load HuggingFace model with 4-bit quantization"""
    q4 = BitsAndBytesConfig(
        load_in_4bit=True,
    )

    llm = HuggingFaceLLM(
        model_name=model_name,
        tokenizer_name=model_name,
        max_new_tokens=512,
        model_kwargs={"quantization_config": q4},
        generate_kwargs={'do_sample': False,},
        device_map="auto",
    )
    return llm

def build_retriever(index):
    """Build enhanced retriever with more context"""
    retriever = index.as_retriever(
        similarity_top_k=6,
    )
    return retriever

# Custom prompt template for enhanced reasoning
CAREER_ADVISOR_PROMPT = PromptTemplate(
    """You are a professional career advisor. Analyze the candidate objectively and provide honest assessments.

Context: {context_str}
Question: {query_str}

Assessment Rules:
- For roles matching their background: Explain why they're suitable with evidence
- For non-matching roles: Be honest about limited fit but suggest why
- Always provide reasoning - never say "cannot answer"
- Give specific examples from their experience
- Keep responses concise (3-4 sentences max)
- Reason well from over the whole context about the question and then answer
- If answer not found in context, infer cautiously and provide insights

Response format:
- Direct answer and brief reasoning with evidence
- If poor fit: suggest better alternatives

Answer:"""
)




def load_qa_engine(retriever, llm, memory):
    """Create enhanced query engine with memory support - FIXED VERSION"""

    # ✅ CORRECT: Use from_args() factory method that supports memory
    qa_engine = RetrieverQueryEngine.from_args(
        retriever=retriever,
        llm=llm,
        memory=memory,
        response_mode="tree_summarize",
        verbose=False
    )

    # Apply custom prompt template for enhanced reasoning
    qa_engine.update_prompts({
        "response_synthesizer:summary_template": CAREER_ADVISOR_PROMPT
    })

    return qa_engine

def load_index(persist_dir='resume_index'):
    """Load persisted index"""
    storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
    index = load_index_from_storage(storage_context)
    return index

def build_agent(model_name, memory):
    """Build complete AI agent with enhanced reasoning capabilities"""
    print("🔄 Loading model...")
    llm = load_model(model_name)

    print("📂 Loading index...")
    index = load_index()

    print("🔍 Building retriever...")
    retriever = build_retriever(index)

    print("🤖 Creating enhanced QA engine...")
    qa = load_qa_engine(retriever, llm, memory)

    print("✅ Agent ready with enhanced reasoning!")
    return qa

# Usage
enhanced_qa = build_agent(model_name, memory)


🔄 Loading model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

📂 Loading index...
Loading llama_index.core.storage.kvstore.simple_kvstore from resume_index/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from resume_index/index_store.json.
🔍 Building retriever...
🤖 Creating enhanced QA engine...
✅ Agent ready with enhanced reasoning!


In [17]:
# === SUPPRESS ALL WARNINGS ===
import warnings
import os
import logging
import time
from typing import Dict, List

# Suppress all Python warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

# Suppress transformers/HuggingFace warnings
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Suppress PyTorch warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

# Suppress LlamaIndex warnings
os.environ["LLAMA_INDEX_CACHE_DIR"] = "/tmp/llama_index_cache"
logging.getLogger("llama_index").setLevel(logging.ERROR)

# Suppress other common ML library warnings
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)


def test_resume_qa_system(qa_engine, index=None):
    """
    Enhanced testing suite for resume Q&A system with non-discriminative language
    Tests the enhanced reasoning capabilities and response quality
    """

    print("🚀 Enhanced Resume Q&A System Testing")
    print("✨ Testing with improved reasoning and analysis capabilities")
    print("=" * 50)

    results = {
        'total_tests': 0,
        'passed': 0,
        'failed': 0,
        'performance': {},
        'section_coverage': {},
        'quality_scores': {},
        'reasoning_quality': {}
    }

    # === UPDATED CORE TEST QUESTIONS (Non-discriminative) ===
    test_suite = {
        'Basics': [
            "What is the candidate's email address?",
            "Where is the candidate located?",
            "What are the candidate's contact details?"
        ],
        'Skills': [
            "What programming languages does the candidate know?",
            "What AI/ML technologies has the candidate used?",
            "What cloud platforms has the candidate worked with?"
        ],
        'Experience': [
            "What companies has the candidate worked at?",
            "What was the candidate's role at TechCorp Solutions?",
            "How many years of experience does the candidate have?"
        ],
        'Education': [
            "What degrees does the candidate have?",
            "Where did the candidate study?",
            "What is the candidate's educational background?"
        ],
        'Certifications': [
            "What certifications does the candidate have?",
            "Does the candidate have AWS certification?",
            "What professional certifications are listed?"
        ]
    }

    # === ENHANCED QUALITY VALIDATION ===
    quality_checks = {
        "What certifications does the candidate have?":
            ["AWS", "Google Cloud", "Kubernetes", "TensorFlow", "MongoDB"],
        "What programming languages does the candidate know?":
            ["Python", "Java", "Scala", "Go", "JavaScript"],
        "What companies has the candidate worked at?":
            ["TechCorp", "DataFlow", "InnovateLabs", "CloudTech"]
    }

    # === REASONING QUALITY TESTS (New) ===
    reasoning_tests = {
        "Is the candidate suitable for data analyst roles?": {
            "should_contain": ["suitable", "qualified", "experience", "skills"],
            "should_not_contain": ["cannot answer", "does not mention", "text does not contain"],
            "reasoning_required": True
        },
        "What are the candidate's core strengths?": {
            "should_contain": ["strength", "experience", "expertise", "skill"],
            "should_not_contain": ["cannot", "does not"],
            "reasoning_required": True
        }
    }

    # === RUN CORE TESTS ===
    all_response_times = []
    section_success = {}

    for section, questions in test_suite.items():
        print(f"\n📋 Testing {section}")
        section_success[section] = {'passed': 0, 'total': len(questions)}

        for question in questions:
            results['total_tests'] += 1

            try:
                # Time the response
                start_time = time.time()
                response = qa_engine.query(question)
                response_time = time.time() - start_time
                all_response_times.append(response_time)

                # Enhanced validation
                answer = str(response)
                is_valid = (
                    len(answer) > 20 and  # Longer minimum for quality responses
                    "sorry" not in answer.lower() and
                    "cannot answer" not in answer.lower() and
                    "does not mention" not in answer.lower()
                )

                if is_valid:
                    results['passed'] += 1
                    section_success[section]['passed'] += 1
                    status = "✅"
                else:
                    results['failed'] += 1
                    status = "⚠️"

                print(f"  {status} {question[:45]}... ({response_time:.2f}s)")

            except Exception as e:
                results['failed'] += 1
                print(f"  ❌ {question[:45]}... Error: {str(e)[:30]}")

    # === QUALITY SCORING ===
    print(f"\n⭐ Quality Analysis")
    quality_scores = []

    for question, expected_keywords in quality_checks.items():
        try:
            response = qa_engine.query(question)
            answer = str(response).lower()

            found = [kw for kw in expected_keywords if kw.lower() in answer]
            score = (len(found) / len(expected_keywords)) * 100
            quality_scores.append(score)

            print(f"  📊 {question[:35]}... Score: {score:.0f}%")

        except Exception as e:
            quality_scores.append(0)
            print(f"  ❌ Quality check failed: {e}")

    # === REASONING QUALITY TESTS (New) ===
    print(f"\n🧠 Reasoning Quality Analysis")
    reasoning_scores = []

    for question, criteria in reasoning_tests.items():
        try:
            response = qa_engine.query(question)
            answer = str(response).lower()

            # Check positive criteria
            positive_score = sum(1 for term in criteria["should_contain"] if term in answer)
            positive_ratio = positive_score / len(criteria["should_contain"])

            # Check negative criteria (should be 0)
            negative_score = sum(1 for term in criteria["should_not_contain"] if term in answer)
            negative_penalty = negative_score / len(criteria["should_not_contain"])

            # Calculate overall reasoning score
            reasoning_score = max(0, (positive_ratio - negative_penalty) * 100)
            reasoning_scores.append(reasoning_score)

            status = "🧠" if reasoning_score >= 70 else "⚠️"
            print(f"  {status} {question[:35]}... Reasoning: {reasoning_score:.0f}%")

        except Exception as e:
            reasoning_scores.append(0)
            print(f"  ❌ Reasoning test failed: {e}")

    # === PERFORMANCE METRICS ===
    if all_response_times:
        results['performance'] = {
            'avg_response_time': sum(all_response_times) / len(all_response_times),
            'max_response_time': max(all_response_times),
            'min_response_time': min(all_response_times)
        }

    # === INDEX VALIDATION ===
    if index:
        try:
            node_count = len(index.docstore.docs)
            sections_found = set()
            for node in index.docstore.docs.values():
                if 'section' in node.metadata:
                    sections_found.add(node.metadata['section'])

            results['index_stats'] = {
                'total_nodes': node_count,
                'sections_found': list(sections_found)
            }
        except Exception as e:
            results['index_error'] = str(e)

    # === ENHANCED FINAL REPORT ===
    print(f"\n📊 COMPREHENSIVE TEST SUMMARY")
    print("=" * 35)

    success_rate = (results['passed'] / results['total_tests']) * 100 if results['total_tests'] > 0 else 0
    avg_quality = sum(quality_scores) / len(quality_scores) if quality_scores else 0
    avg_reasoning = sum(reasoning_scores) / len(reasoning_scores) if reasoning_scores else 0

    print(f"🎯 Success Rate: {success_rate:.1f}% ({results['passed']}/{results['total_tests']})")
    print(f"⭐ Quality Score: {avg_quality:.1f}%")
    print(f"🧠 Reasoning Score: {avg_reasoning:.1f}%")

    if all_response_times:
        print(f"⚡ Avg Response: {results['performance']['avg_response_time']:.2f}s")

    if index:
        print(f"📂 Index Nodes: {results['index_stats']['total_nodes']}")
        print(f"📋 Sections: {', '.join(results['index_stats']['sections_found'])}")

    # Section breakdown
    print(f"\n📂 Section Performance:")
    for section, stats in section_success.items():
        section_rate = (stats['passed'] / stats['total']) * 100
        print(f"  {section:12}: {section_rate:.0f}% ({stats['passed']}/{stats['total']})")

    # Enhanced system health assessment
    overall_score = (success_rate + avg_quality + avg_reasoning) / 3

    if overall_score >= 80 and avg_reasoning >= 70:
        print(f"\n🟢 System Status: EXCELLENT (Enhanced reasoning working)")
    elif overall_score >= 70 and success_rate >= 80:
        print(f"\n🟡 System Status: GOOD (Some reasoning improvements needed)")
    elif success_rate >= 60:
        print(f"\n🟠 System Status: NEEDS ATTENTION (Reasoning capabilities limited)")
    else:
        print(f"\n🔴 System Status: CRITICAL ISSUES (Major problems detected)")

    # Store reasoning results
    results['reasoning_quality'] = {
        'avg_reasoning_score': avg_reasoning,
        'reasoning_tests_passed': sum(1 for score in reasoning_scores if score >= 70),
        'total_reasoning_tests': len(reasoning_scores)
    }

    return results


# === ENHANCED USAGE ===
# Use with your enhanced agent instead of basic qa
if 'enhanced_qa' in globals():
    print("🚀 Testing with Enhanced QA Agent")
    test_results = test_resume_qa_system(enhanced_qa, index)
else:
    print("⚠️ Using basic QA engine - consider using enhanced_qa for better results")
    test_results = test_resume_qa_system(qa, index)

# Enhanced results saving
import json
enhanced_results = {
    **test_results,
    'test_metadata': {
        'test_version': 'enhanced_v2.0',
        'reasoning_enabled': True,
        'non_discriminative': True,
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }
}

# Optional: Save enhanced results
# with open('enhanced_test_results.json', 'w') as f:
#     json.dump(enhanced_results, f, indent=2)


🚀 Testing with Enhanced QA Agent
🚀 Enhanced Resume Q&A System Testing
✨ Testing with improved reasoning and analysis capabilities

📋 Testing Basics
  ✅ What is the candidate's email address?... (22.05s)
  ✅ Where is the candidate located?... (23.06s)
  ✅ What are the candidate's contact details?... (17.82s)

📋 Testing Skills
  ✅ What programming languages does the candidate... (22.67s)
  ✅ What AI/ML technologies has the candidate use... (23.56s)
  ✅ What cloud platforms has the candidate worked... (20.97s)

📋 Testing Experience
  ✅ What companies has the candidate worked at?... (48.19s)
  ✅ What was the candidate's role at TechCorp Sol... (20.81s)
  ✅ How many years of experience does the candida... (25.08s)

📋 Testing Education
  ✅ What degrees does the candidate have?... (21.45s)
  ✅ Where did the candidate study?... (23.26s)
  ✅ What is the candidate's educational backgroun... (23.49s)

📋 Testing Certifications
  ✅ What certifications does the candidate have?... (28.85s)
  ✅ Does t

In [18]:
# === SUPPRESS ALL WARNINGS ===
import warnings
import os
import logging

# Suppress all Python warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

# Suppress transformers/HuggingFace warnings
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Suppress PyTorch warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

# Suppress LlamaIndex warnings
os.environ["LLAMA_INDEX_CACHE_DIR"] = "/tmp/llama_index_cache"
logging.getLogger("llama_index").setLevel(logging.ERROR)

# Suppress other common ML library warnings
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)


def detect_interaction_type(query):
    """Detect what type of interaction the user wants"""

    query_lower = query.lower().strip()

    # Greetings and casual interactions
    greetings = ['hi', 'hello', 'hey', 'good morning', 'good afternoon', 'good evening']
    casual_responses = ['thanks', 'thank you', 'ok', 'okay', 'cool', 'nice', 'great']

    if query_lower in greetings:
        return 'greeting'
    elif query_lower in casual_responses:
        return 'acknowledgment'
    elif len(query.split()) <= 2 and query_lower in ['help', '?', 'info']:
        return 'help'
    else:
        return 'question'


def get_humane_response(query, interaction_type, qa_engine):
    """Generate appropriate responses based on interaction type"""

    if interaction_type == 'greeting':
        return """👋 Hello! I'm here to help you learn about the candidate's background and qualifications.

I can answer questions about their:
• Technical skills and expertise
• Work experience and achievements
• Education and certifications
• Suitability for specific roles

What would you like to know about the candidate?"""

    elif interaction_type == 'acknowledgment':
        return "😊 You're welcome! Feel free to ask me anything else about the candidate's resume."

    elif interaction_type == 'help':
        return """💡 Here are some example questions you can ask:

**Skills & Experience:**
• "What programming languages does the candidate know?"
• "Tell me about their AI/ML experience"
• "What cloud platforms have they worked with?"

**Career & Suitability:**
• "Is the candidate suitable for a data scientist role?"
• "What are their core strengths?"
• "What leadership experience do they have?"

**Specific Details:**
• "Where did the candidate study?"
• "What certifications do they have?"
• "What companies have they worked at?"

Just ask naturally - I'll understand! 🤖"""

    else:  # It's a real question - SIMPLIFIED (no prompt engineering)
        # ✅ MODIFIED: Direct query without additional prompting
        # Since prompt engineering is handled in model loading
        return qa_engine.query(query)


def qa_assistant(query, qa_engine):
    """More humane QA assistant that recognizes different interaction types"""

    # Check for exit commands
    if query.lower().strip() in ['exit', 'quit', 'stop', 'q', 'bye', 'goodbye']:
        return False

    # Handle empty input
    if not query.strip():
        print("❓ Please enter a question or just say 'hi' to get started!")
        return True

    # Detect interaction type and respond appropriately
    try:
        interaction_type = detect_interaction_type(query)

        if interaction_type in ['greeting', 'acknowledgment', 'help']:
            print("💭 ...")  # Brief thinking indicator for casual interactions
        else:
            print("🔍 Analyzing the candidate's background...")

        response = get_humane_response(query, interaction_type, qa_engine)
        print(f"🤖 {response}")
        return True

    except Exception as e:
        print(f"❌ Sorry, I encountered an error: {e}")
        print("💡 Try rephrasing your question or type 'help' for examples.")
        return True


def start_qa_session(qa_engine):
    """Start a more humane interactive QA session"""
    print("🤖 Resume Q&A Assistant")
    print("✨ Ask me anything about the candidate's background!")
    print("💬 Just say 'hi' to get started, or ask any question directly")
    print("🚪 Type 'quit' to exit\n")

    question_count = 0
    greeting_given = False

    while True:
        try:
            user_question = input("💬 You: ")

            # Track if user has been greeted
            if not greeting_given and detect_interaction_type(user_question) != 'greeting':
                greeting_given = True

            if not qa_assistant(user_question, qa_engine):
                if question_count == 0:
                    print(f"\n👋 Thanks for stopping by!")
                elif question_count == 1:
                    print(f"\n📊 Hope that was helpful! Feel free to come back anytime.")
                else:
                    print(f"\n📊 Great questions! We covered {question_count} topics about the candidate.")
                break

            # Only count substantive questions
            if detect_interaction_type(user_question) == 'question':
                question_count += 1

        except KeyboardInterrupt:
            print("\n\n⚠️ Session interrupted. Have a great day! 👋")
            break


# Usage
start_qa_session(enhanced_qa)


🤖 Resume Q&A Assistant
✨ Ask me anything about the candidate's background!
💬 Just say 'hi' to get started, or ask any question directly
🚪 Type 'quit' to exit

💬 You: are they suitable for data analyst?
🔍 Analyzing the candidate's background...
🤖 

The provided text does not describe any roles related to data analysis, therefore I cannot provide an assessment on whether Alexandra Chen is suitable for such roles.

However, based on her experience and skills, she appears to be well-suited for positions involving AI/ML engineering, software engineering, or data science. Her strong technical background, expertise in various programming languages and frameworks, and ability to work on complex projects make her a strong candidate for these roles.
💬 You: what do you think based on her skills?
🔍 Analyzing the candidate's background...
🤖 

Alexandra Chen has a strong background in AI/ML engineering and backend development, making her well-suited for roles in various fields. Her expertise in Pyth

# Resume Analysis and Q&A System

This notebook implements a system for extracting information from a resume PDF, indexing the extracted data, and providing a conversational Q&A interface to query the resume's content.

## System Components

1.  **Dependency Installation**:
    -   Installs necessary libraries including `llama-index`, `sentence-transformers`, `bitsandbytes`, and specific LlamaIndex integrations for HuggingFace models.

2.  **Llama Cloud Extractor**:
    -   Utilizes `llama_cloud_services.LlamaExtract` to load a pre-configured extraction agent (`Precise_Resume_extract`) using a provided project ID and API key.
    -   This agent is designed to parse structured information (like skills, experience, education) from resume documents.
    -   The loaded agent's data schema is printed to show the expected output structure.

3.  **Resume Data Extraction**:
    -   The loaded `extractor` agent is used to process a specific resume file (`/content/Backend AI Engineer Resume Template - Claude.pdf`).
    -   The extracted structured data (in JSON format) is stored in the `parsed_data` variable and printed.

4.  **JSON to Text Conversion**:
    -   Custom Python functions (`flatten_json` and `resume_json_to_text`) are defined to convert the nested JSON output from the extractor into a flattened, human-readable markdown-like text format.
    -   This converted text (`resume_text`) is then printed.

5.  **Text Sectioning**:
    -   A function `extract_sections` is used to split the `resume_text` into logical sections based on markdown headers (e.g., "Basics", "Skills", "Experience").
    -   This creates a dictionary (`sections`) where keys are section titles and values are the text content of each section.

6.  **Embedding Model Setup**:
    -   Configures the embedding model used by LlamaIndex for creating vector representations of the text data.
    -   The `sentence-transformers/all-MiniLM-L6-v2` model is set as the default embedding model in `Settings`.

7.  **Node Creation with Custom Chunking**:
    -   The `create_nodes_with_custom_chunking` function processes the text `sections`.
    -   For each section, it creates a `Document` object with the section content and metadata (the section name).
    -   It uses a `SentenceSplitter` configured to keep each section as a single node by setting a large `chunk_size`.
    -   This results in a list of `nodes`, where each node represents a distinct section of the resume.
    -   Node metadata and text previews are printed for verification.

8.  **Vector Store Indexing**:
    -   A `VectorStoreIndex` is created directly from the list of `nodes`. This indexes the content of each resume section, making it searchable based on vector similarity.
    -   The index is then persisted to disk in the `resume_index` directory for later loading.

9.  **Index Verification and Analysis**:
    -   Code cells verify the basic properties of the created index (type, node count, ID).
    -   The `analyze_index_contents` function provides a more detailed breakdown of the index, including total characters, average characters per node, and statistics for each section.

10. **Index Loading**:
    -   Demonstrates how to load the persisted index from the `resume_index` directory using `StorageContext`.
    -   Verification is performed to ensure the loaded index has the correct number of nodes.

11. **Hugging Face LLM Loading**:
    -   A function `load_model` is defined to load a Hugging Face Language Model (`google/gemma-7b-it`) using `HuggingFaceLLM`.
    -   4-bit quantization (`BitsAndBytesConfig`) is applied to reduce memory usage.

12. **Retriever and Query Engine Setup**:
    -   A `retriever` is built from the loaded `index` to fetch relevant nodes based on a query (`similarity_top_k=6`).
    -   A custom `CAREER_ADVISOR_PROMPT` is defined to guide the LLM's response, instructing it to act as a career advisor, provide reasoning, and adhere to specific formatting rules.
    -   A `ChatMemoryBuffer` is initialized to maintain conversation history.
    -   The `load_qa_engine` function creates a `RetrieverQueryEngine` from the retriever, LLM, and memory, and updates the response synthesis prompt with the `CAREER_ADVISOR_PROMPT`.

13. **Agent Building**:
    -   The `build_agent` function orchestrates the loading of the model, index, retriever, and query engine, creating a complete AI agent ready for interaction.

14. **Automated Testing Suite**:
    -   The `test_resume_qa_system` function provides a comprehensive evaluation of the QA system.
    -   It runs a set of predefined questions across different resume sections.
    -   Includes checks for successful response generation, quality scoring based on expected keywords, and a novel "reasoning quality" assessment to evaluate how well the agent adheres to the career advisor prompt's instructions for providing justified answers.
    -   Prints a detailed summary of test results, including success rates, quality scores, reasoning scores, performance metrics, and index statistics.

15. **Interactive QA Assistant**:
    -   The `qa_assistant` and `start_qa_session` functions provide a user-friendly command-line interface.
    -   `detect_interaction_type` identifies if the user input is a greeting, acknowledgment, help request, or a question.
    -   `get_humane_response` provides tailored responses for casual interactions or uses the `qa_engine` for questions.
    -   The `start_qa_session` function loops, taking user input and using the `qa_assistant` to provide responses until the user types an exit command.
    -   This allows for natural language interaction with the indexed resume content.

This system demonstrates a complete pipeline for processing unstructured resume data, structuring it, indexing it for efficient retrieval, and building a conversational interface with enhanced reasoning capabilities using LlamaIndex and Hugging Face models.